# =========================================================
# UNIQA Group Operations - Medical Invoice Data Extraction
# Christoph Häusler | 19.02.2026
# =========================================================

# Assumptions
- invoice language = german
- invoice file format = pdf (digital, pictures, scans)

# Goals
- extract following data:
    - Date of invoice
    - Name of doctor
    - Medical speciality
    - Diagnosis
    - Prescribed treatment
    - Value of indemnity
    - ?

--- 

# Process Overview
Read -> Classification -> Processing -> Validation

## Read
libraries:
- pymupdf

## Classification
classify pdf-files in different types:
- digital
- scan/picture
    - machinewriting
    - handwriting

each class has a different processing pipepline

## Processing
libraries:
- ollama
    - glm-ocr (for OCR)
    - qwen3 (for llm tasks)

digital -> read via pyumpdf -> 
machine writing -> ocr + llm(layout?)
handwriting -> ocr + llm

## Validation
- sum = sum of all other numbers
- predifined field contents (address, type of medical, SVNR lenght, birthdates, names?)


-> manuel user validation of uncertain data (confidence?)

In [ ]:
# libraries and constants
import os
import json
import pymupdf
import pymupdf.layout
import pymupdf4llm
import ollama
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv

# Configuration
INVOICES_DIR = Path("../invoices")
INVOICES_DIR.mkdir(exist_ok=True)

OCR_MODEL = "glm-ocr"
LLM_MODEL = "qwen3:8b"
LLM_MODEL_LIGHT = "qwen3:4b"

In [ ]:
# Generate synthetic invoices (pdf/pdf_scans)

from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import cm
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.enums import TA_RIGHT, TA_LEFT, TA_CENTER
from PIL import Image as PILImage
import numpy as np
import cv2
import pymupdf
import random
from datetime import datetime, timedelta
import io

# ── Sample data ────────────────────────────────────────────────────────────────

doctors = [
    {"name": "Dr. Elisabeth Gruber", "specialty": "Allgemeinmedizin", "address": "Mariahilfer Straße 45, 1060 Wien", "phone": "+43 1 234 5678", "uid": "ATU12345678", "template": 0, "color": colors.HexColor("#003366")},
    {"name": "Dr. Thomas Bauer", "specialty": "Innere Medizin", "address": "Ringstraße 12, 4020 Linz", "phone": "+43 732 987 654", "uid": "ATU23456789", "template": 1, "color": colors.HexColor("#1a1a1a")},
    {"name": "Dr. Sabine Hofer", "specialty": "Dermatologie", "address": "Getreidegasse 8, 5020 Salzburg", "phone": "+43 662 445 566", "uid": "ATU34567890", "template": 2, "color": colors.HexColor("#2c5f2e")},
    {"name": "Dr. Michael Schwarz", "specialty": "Orthopädie", "address": "Herrengasse 3, 8010 Graz", "phone": "+43 316 772 839", "uid": "ATU45678901", "template": 0, "color": colors.HexColor("#7b2d00")},
    {"name": "Dr. Anna Berger", "specialty": "Gynäkologie", "address": "Museumstraße 22, 6020 Innsbruck", "phone": "+43 512 334 455", "uid": "ATU56789012", "template": 1, "color": colors.HexColor("#4a4a8a")},
    {"name": "Dr. Klaus Müller", "specialty": "Neurologie", "address": "Landstraße 67, 4020 Linz", "phone": "+43 732 556 677", "uid": "ATU67890123", "template": 2, "color": colors.HexColor("#1a1a1a")},
    {"name": "Dr. Maria Steininger", "specialty": "Psychiatrie", "address": "Operngasse 14, 1010 Wien", "phone": "+43 1 876 5432", "uid": "ATU78901234", "template": 0, "color": colors.HexColor("#2c5f2e")},
]

patients = [
    {"name": "Maria Huber", "dob": "15.03.1978", "svnr": "1234 150378"},
    {"name": "Josef Mayr", "dob": "22.07.1965", "svnr": "5678 220765"},
    {"name": "Sandra Fuchs", "dob": "08.11.1990", "svnr": "9012 081190"},
    {"name": "Peter Wagner", "dob": "30.01.1955", "svnr": "3456 300155"},
    {"name": "Christine Leitner", "dob": "17.06.1983", "svnr": "7890 170683"},
    {"name": "Andreas Brandner", "dob": "04.09.1972", "svnr": "2345 040972"},
    {"name": "Katharina Wiesner", "dob": "28.12.1988", "svnr": "6789 281288"},
    {"name": "Markus Pichler", "dob": "11.04.1960", "svnr": "0123 110460"},
    {"name": "Eva Sonnleitner", "dob": "19.08.1995", "svnr": "4567 190895"},
    {"name": "Robert Kastner", "dob": "07.02.1948", "svnr": "8901 070248"},
]

treatments = {
    "Allgemeinmedizin": [
        ("Erstuntersuchung", "Ö1", 85.00),
        ("Kontrolluntersuchung", "Ö2", 55.00),
        ("Blutabnahme", "L100", 18.50),
        ("EKG", "E110", 42.00),
    ],
    "Innere Medizin": [
        ("Internistische Untersuchung", "I200", 120.00),
        ("Ultraschall Abdomen", "U300", 95.00),
        ("Langzeit-EKG", "E220", 110.00),
    ],
    "Dermatologie": [
        ("Hautuntersuchung", "D100", 90.00),
        ("Kryotherapie", "D220", 75.00),
        ("Dermatoskopie", "D150", 65.00),
    ],
    "Orthopädie": [
        ("Orthopädische Untersuchung", "O100", 110.00),
        ("Infiltration", "O210", 85.00),
        ("Röntgenbefundung", "R100", 55.00),
    ],
    "Gynäkologie": [
        ("Gynäkologische Untersuchung", "G100", 100.00),
        ("Ultraschall", "G210", 80.00),
        ("Zytologie", "G150", 60.00),
    ],
    "Neurologie": [
        ("Neurologische Untersuchung", "N100", 130.00),
        ("EEG", "N220", 115.00),
        ("EMG", "N230", 125.00),
    ],
    "Psychiatrie": [
        ("Psychiatrische Erstuntersuchung", "P100", 150.00),
        ("Psychotherapeutische Sitzung", "P210", 110.00),
        ("Medikamentöse Beratung", "P150", 90.00),
    ],
}

diagnoses = {
    "Allgemeinmedizin": ["Hypertonie (I10)", "Diabetes mellitus Typ 2 (E11)", "Akute Bronchitis (J20)"],
    "Innere Medizin": ["Gastritis (K29)", "Herzinsuffizienz (I50)", "Anämie (D64)"],
    "Dermatologie": ["Ekzem (L30)", "Psoriasis (L40)", "Seborrhoisches Ekzem (L21)"],
    "Orthopädie": ["Lumbalgie (M54.5)", "Gonarthrose (M17)", "Schulterimpingement (M75.1)"],
    "Gynäkologie": ["Dysmenorrhoe (N94.4)", "Myom (D25)", "Zervizitis (N72)"],
    "Neurologie": ["Migräne (G43)", "Polyneuropathie (G62)", "Schwindel (R42)"],
    "Psychiatrie": ["Depressive Episode (F32)", "Angststörung (F41)", "Burnout (Z73)"],
}

document_titles = ["Rechnung", "Honorarnote", "Arzthonorar", "Privatrechnung"]

header_colors = [
    colors.HexColor("#003366"),
    colors.HexColor("#1a1a1a"),
    colors.HexColor("#2c5f2e"),
    colors.HexColor("#7b2d00"),
    colors.HexColor("#4a4a8a"),
]


def random_date():
    start = datetime(2025, 1, 1)
    end = datetime(2025, 12, 31)
    return start + timedelta(days=random.randint(0, (end - start).days))


def random_iban():
    return f"AT{random.randint(10,99)} {random.randint(1000,9999)} {random.randint(1000,9999)} {random.randint(1000,9999)} {random.randint(1000,9999)}"


# ── Template A – Modern ─────────────────────────────────────────────────────────

def build_template_a(story, invoice_data, styles, header_color):
    bold = ParagraphStyle("bold", fontSize=10, fontName="Helvetica-Bold")
    normal = ParagraphStyle("normal", fontSize=9, fontName="Helvetica", spaceAfter=3)
    sub = ParagraphStyle("sub", fontSize=8, fontName="Helvetica", textColor=colors.grey, spaceAfter=2)
    title_style = ParagraphStyle("title", fontSize=16, fontName="Helvetica-Bold", textColor=header_color)

    story.append(Paragraph(invoice_data["title"], title_style))
    story.append(Spacer(1, 0.3*cm))
    story.append(Paragraph(invoice_data["doctor"]["name"], ParagraphStyle("dname", fontSize=12, fontName="Helvetica-Bold")))
    story.append(Paragraph(invoice_data["doctor"]["specialty"], sub))
    story.append(Paragraph(invoice_data["doctor"]["address"], sub))
    story.append(Paragraph(invoice_data["doctor"]["phone"], sub))
    if invoice_data.get("show_uid"):
        story.append(Paragraph(f"UID: {invoice_data['doctor']['uid']}", sub))
    story.append(Spacer(1, 0.4*cm))

    story.append(Table([[""]], colWidths=[17*cm], style=TableStyle([
        ("LINEBELOW", (0,0), (-1,-1), 1, header_color)
    ])))
    story.append(Spacer(1, 0.4*cm))

    info_data = [
        ["Patient:", invoice_data["patient"]["name"], "Nr.:", invoice_data["invoice_no"]],
        ["Geb.datum:", invoice_data["patient"]["dob"], "Datum:", invoice_data["date"]],
        ["SVNR:", invoice_data["patient"]["svnr"], "", ""],
    ]
    info_table = Table(info_data, colWidths=[3*cm, 6*cm, 3*cm, 5*cm])
    info_table.setStyle(TableStyle([
        ("FONTNAME", (0,0), (0,-1), "Helvetica-Bold"),
        ("FONTNAME", (2,0), (2,-1), "Helvetica-Bold"),
        ("FONTSIZE", (0,0), (-1,-1), 9),
        ("BOTTOMPADDING", (0,0), (-1,-1), 4),
    ]))
    story.append(info_table)
    story.append(Spacer(1, 0.5*cm))

    story.append(Paragraph(f"Diagnose: {invoice_data['diagnosis']}", bold))
    story.append(Spacer(1, 0.4*cm))

    table_data = [["Pos.", "Leistung", "Betrag"]]
    for t in invoice_data["treatments"]:
        table_data.append([t["code"], t["description"], f"€ {t['amount']:.2f}"])
    total = sum(t["amount"] for t in invoice_data["treatments"])
    table_data.append(["", "Gesamt", f"€ {total:.2f}"])

    t_table = Table(table_data, colWidths=[2.5*cm, 11*cm, 3.5*cm])
    t_table.setStyle(TableStyle([
        ("BACKGROUND", (0,0), (-1,0), header_color),
        ("TEXTCOLOR", (0,0), (-1,0), colors.white),
        ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
        ("FONTSIZE", (0,0), (-1,-1), 9),
        ("ROWBACKGROUNDS", (0,1), (-1,-2), [colors.white, colors.HexColor("#f5f5f5")]),
        ("FONTNAME", (0,-1), (-1,-1), "Helvetica-Bold"),
        ("LINEABOVE", (0,-1), (-1,-1), 0.5, colors.grey),
        ("ALIGN", (2,0), (2,-1), "RIGHT"),
        ("BOTTOMPADDING", (0,0), (-1,-1), 5),
        ("TOPPADDING", (0,0), (-1,-1), 5),
    ]))
    story.append(t_table)
    story.append(Spacer(1, 0.8*cm))

    if invoice_data.get("payment_method") == "bank":
        story.append(Paragraph("Bitte überweisen Sie den Betrag auf folgendes Konto:", normal))
        story.append(Paragraph(f"IBAN: {invoice_data['iban']}", bold))
        story.append(Paragraph("Zahlungsziel: 14 Tage", normal))
    else:
        story.append(Paragraph("Betrag dankend erhalten (Barzahlung).", normal))

    story.append(Spacer(1, 1*cm))
    story.append(Paragraph("Mit freundlichen Grüßen", sub))
    story.append(Paragraph(invoice_data["doctor"]["name"], normal))

    return total


# ── Template B – Classic ────────────────────────────────────────────────────────

def build_template_b(story, invoice_data, styles, header_color):
    normal = ParagraphStyle("normal", fontSize=9, fontName="Helvetica", spaceAfter=3)
    bold = ParagraphStyle("bold", fontSize=9, fontName="Helvetica-Bold")
    title_style = ParagraphStyle("title", fontSize=13, fontName="Helvetica-Bold", spaceAfter=6)
    small = ParagraphStyle("small", fontSize=8, fontName="Helvetica", textColor=colors.grey)

    # Doctor block top right style – simulate letter layout
    doctor_block = [
        [Paragraph(invoice_data["doctor"]["name"], bold), Paragraph(invoice_data["title"], title_style)],
        [Paragraph(invoice_data["doctor"]["specialty"], small), Paragraph(f"Nr: {invoice_data['invoice_no']}", normal)],
        [Paragraph(invoice_data["doctor"]["address"], small), Paragraph(f"Datum: {invoice_data['date']}", normal)],
        [Paragraph(invoice_data["doctor"]["phone"], small), ""],
    ]
    header_table = Table(doctor_block, colWidths=[9*cm, 8*cm])
    header_table.setStyle(TableStyle([
        ("VALIGN", (0,0), (-1,-1), "TOP"),
        ("BOTTOMPADDING", (0,0), (-1,-1), 3),
    ]))
    story.append(header_table)
    story.append(Spacer(1, 0.5*cm))

    story.append(Table([[""]], colWidths=[17*cm], style=TableStyle([
        ("LINEBELOW", (0,0), (-1,-1), 0.5, colors.black)
    ])))
    story.append(Spacer(1, 0.4*cm))

    story.append(Paragraph(f"Patient: {invoice_data['patient']['name']}  |  Geb.: {invoice_data['patient']['dob']}  |  SVNR: {invoice_data['patient']['svnr']}", normal))
    story.append(Spacer(1, 0.3*cm))
    story.append(Paragraph(f"Diagnose: {invoice_data['diagnosis']}", bold))
    story.append(Spacer(1, 0.4*cm))

    table_data = [["Code", "Bezeichnung", "Honorar"]]
    for t in invoice_data["treatments"]:
        table_data.append([t["code"], t["description"], f"EUR {t['amount']:.2f}"])
    total = sum(t["amount"] for t in invoice_data["treatments"])
    table_data.append(["", "Gesamtbetrag:", f"EUR {total:.2f}"])

    t_table = Table(table_data, colWidths=[2.5*cm, 11*cm, 3.5*cm])
    t_table.setStyle(TableStyle([
        ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
        ("LINEBELOW", (0,0), (-1,0), 0.5, colors.black),
        ("LINEABOVE", (0,-1), (-1,-1), 0.5, colors.black),
        ("FONTNAME", (0,-1), (-1,-1), "Helvetica-Bold"),
        ("FONTSIZE", (0,0), (-1,-1), 9),
        ("ALIGN", (2,0), (2,-1), "RIGHT"),
        ("BOTTOMPADDING", (0,0), (-1,-1), 5),
        ("TOPPADDING", (0,0), (-1,-1), 5),
        ("ROWBACKGROUNDS", (0,1), (-1,-2), [colors.white, colors.HexColor("#eeeeee")]),
    ]))
    story.append(t_table)
    story.append(Spacer(1, 0.8*cm))

    if invoice_data.get("payment_method") == "bank":
        story.append(Paragraph(f"Bankverbindung: {invoice_data['iban']}", normal))
        story.append(Paragraph("Zahlbar innerhalb von 30 Tagen.", small))
    else:
        story.append(Paragraph("Barzahlung erhalten – kein weiterer Zahlungsaufwand.", normal))

    if invoice_data.get("show_uid"):
        story.append(Spacer(1, 0.3*cm))
        story.append(Paragraph(f"Steuernummer/UID: {invoice_data['doctor']['uid']}", small))

    return total


# ── Template C – Minimal ────────────────────────────────────────────────────────

def build_template_c(story, invoice_data, styles, header_color):
    normal = ParagraphStyle("normal", fontSize=10, fontName="Helvetica", spaceAfter=5)
    bold = ParagraphStyle("bold", fontSize=10, fontName="Helvetica-Bold", spaceAfter=3)
    small = ParagraphStyle("small", fontSize=8, fontName="Helvetica", textColor=colors.grey, spaceAfter=2)
    title_style = ParagraphStyle("title", fontSize=11, fontName="Helvetica-Bold", spaceAfter=8)

    story.append(Paragraph(f"{invoice_data['title']} – {invoice_data['invoice_no']}", title_style))
    story.append(Paragraph(f"{invoice_data['doctor']['name']} | {invoice_data['doctor']['specialty']}", normal))
    story.append(Paragraph(f"{invoice_data['doctor']['address']} | Tel: {invoice_data['doctor']['phone']}", small))
    story.append(Spacer(1, 0.6*cm))

    story.append(Paragraph(f"Ausgestellt am: {invoice_data['date']}", normal))
    story.append(Paragraph(f"Patient/in: {invoice_data['patient']['name']}, geb. {invoice_data['patient']['dob']}", normal))
    story.append(Paragraph(f"Sozialversicherungsnummer: {invoice_data['patient']['svnr']}", normal))
    story.append(Spacer(1, 0.4*cm))

    story.append(Paragraph(f"Diagnose: {invoice_data['diagnosis']}", bold))
    story.append(Spacer(1, 0.3*cm))

    total = sum(t["amount"] for t in invoice_data["treatments"])
    for t in invoice_data["treatments"]:
        story.append(Paragraph(f"  {t['code']}  –  {t['description']}  ......  € {t['amount']:.2f}", normal))

    story.append(Spacer(1, 0.3*cm))
    story.append(Table([[""]], colWidths=[17*cm], style=TableStyle([
        ("LINEBELOW", (0,0), (-1,-1), 0.5, colors.black)
    ])))
    story.append(Spacer(1, 0.2*cm))
    story.append(Paragraph(f"Gesamtbetrag: € {total:.2f}", bold))
    story.append(Spacer(1, 0.6*cm))

    if invoice_data.get("payment_method") == "bank":
        story.append(Paragraph(f"IBAN: {invoice_data['iban']}", normal))
    else:
        story.append(Paragraph("Betrag bar erhalten.", normal))

    return total


# ── Scan simulation ─────────────────────────────────────────────────────────────

def simulate_scan(pdf_path: Path):
    doc = pymupdf.open(str(pdf_path))
    page = doc[0]
    mat = pymupdf.Matrix(2.0, 2.0)  # 2x resolution for realism
    pix = page.get_pixmap(matrix=mat)
    img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)

    if pix.n == 4:
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGBA2RGB)

    # Random slight rotation
    angle = random.uniform(-2.5, 2.5)
    h, w = img_array.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    img_array = cv2.warpAffine(img_array, M, (w, h), borderValue=(245, 245, 245))

    # Slight brightness/contrast variation
    alpha = random.uniform(0.88, 1.05)
    beta = random.randint(-8, 8)
    img_array = cv2.convertScaleAbs(img_array, alpha=alpha, beta=beta)

    # Subtle noise
    noise = np.random.normal(0, random.uniform(1, 4), img_array.shape).astype(np.int16)
    img_array = np.clip(img_array.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    # Save back as image-based PDF
    pil_img = PILImage.fromarray(img_array)
    scan_path = pdf_path.parent / pdf_path.name.replace("_digital.pdf", "_scan.pdf")
    pil_img.save(str(scan_path), "PDF", resolution=150)
    doc.close()

    return scan_path


# ── Main generation loop ────────────────────────────────────────────────────────


templates = [build_template_a, build_template_b, build_template_c]
invoices_metadata = []
scan_indices = random.sample(range(10), 5)

# Shuffle both lists so each patient appears exactly once
shuffled_patients = random.sample(patients, len(patients))
shuffled_doctors = random.sample(doctors, len(doctors))

for i in range(10):
    # Cycle through doctors if more invoices than doctors
    doctor = shuffled_doctors[i % len(shuffled_doctors)]
    patient = shuffled_patients[i]
    specialty = doctor["specialty"]
    date = random_date()

    # Each doctor has fixed template + color
    template_fn = templates[doctor["template"]]
    header_color = doctor["color"]

    payment = random.choice(["bank", "bank", "cash"])

    selected_treatments = random.sample(
        treatments[specialty],
        k=random.randint(1, min(3, len(treatments[specialty])))
    )

    invoice_data = {
        "title": random.choice(document_titles),
        "invoice_no": f"RE-{random.randint(1000,9999)}-2025",
        "doctor": doctor,
        "patient": patient,
        "date": date.strftime("%d.%m.%Y"),
        "diagnosis": random.choice(diagnoses[specialty]),
        "treatments": [{"code": t[1], "description": t[0], "amount": t[2]} for t in selected_treatments],
        "payment_method": payment,
        "iban": random_iban() if payment == "bank" else None,
        "show_uid": random.choice([True, False]),
    }

    filename = INVOICES_DIR / f"invoice_{i+1:02d}_digital.pdf"
    doc = SimpleDocTemplate(
        str(filename),
        pagesize=A4,
        rightMargin=2*cm,
        leftMargin=2*cm,
        topMargin=2*cm,
        bottomMargin=2*cm
    )
    story = []
    styles = getSampleStyleSheet()
    total = template_fn(story, invoice_data, styles, header_color)
    doc.build(story)

    is_scan = i in scan_indices
    final_path = filename
    invoice_type = "digital"

    if is_scan:
        final_path = simulate_scan(filename)
        filename.unlink()
        invoice_type = "scan"

    invoices_metadata.append({
        "file": final_path.name,
        "invoice_no": invoice_data["invoice_no"],
        "doctor": doctor["name"],
        "doctor_address": doctor["address"],
        "specialty": specialty,
        "patient": patient["name"],
        "patient_dob": patient["dob"],
        "patient_svnr": patient["svnr"],
        "date": invoice_data["date"],
        "diagnosis": invoice_data["diagnosis"],
        "total": total,
        "type": invoice_type,
        "template": template_fn.__name__,
    })

    print(f"{'🖨️ ' if is_scan else '📄'} invoice_{i+1:02d} | {template_fn.__name__} | {invoice_type} | {doctor['name']} → {patient['name']} | € {total:.2f}")

print(f"\n✅ {len(invoices_metadata)} invoices ready – {sum(1 for x in invoices_metadata if x['type'] == 'scan')} scans, {sum(1 for x in invoices_metadata if x['type'] == 'digital')} digital")

In [ ]:
# Add metadata for handwritten invoice

handwritten_ground_truth = {
    "file": "invoice_11_scan_handwritten.pdf",
    "invoice_no": "RE-8169-2025",
    "doctor": "Dr. Thomas Bauer",
    "doctor_address": "Ringstraße 12, 4020 Linz",
    "specialty": "Innere Medizin",
    "patient": "Josef Mayr",
    "patient_dob": "22.07.1965",
    "patient_svnr": "5678 220765",
    "date": "19.06.2025",
    "diagnosis": "Gastritis",
    "total": 325.00,
    "type": "handwritten",
    "template": None,
}

all_metadata = invoices_metadata + [handwritten_ground_truth]


In [ ]:
# Classification - PDF Type Detection

TEXT_THRESHOLD = 30 # for scan classification

def detect_pdf_type(pdf_path: Path, threshold: int = TEXT_THRESHOLD) -> str:

    doc = pymupdf.open(str(pdf_path))
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()

    char_count = len(text)
    pdf_type = "digital" if char_count >= threshold else "scan"
    
    name = pdf_path.name.ljust(25)
    chars = f"{char_count} chars".ljust(15)

    print(f"{name} {chars} -> {pdf_type}")
    return pdf_type


# Test pdfs in invoices-folder
pdf_files = sorted(INVOICES_DIR.glob("*.pdf"))
print (f"Found {len(pdf_files)} pdf-files")

detected_invoices = []

for pdf_path in pdf_files:
    detected_type = detect_pdf_type(pdf_path)
    detected_invoices.append({
        "file": pdf_path.name,
        "path": pdf_path,
        "type": detected_type,
    })
print("\nTotal Count")
print("Digital: ".ljust(10), sum(1 for x in detected_invoices if x["type"] == "digital"))
print("Scans: ".ljust(10), sum(1 for x in detected_invoices if x["type"] == "scan"))


In [ ]:
# Extract text from pdf file
import base64

def extract_text_digital(pdf_path: Path) -> str:
    return pymupdf4llm.to_markdown(str(pdf_path))

def pdf_to_image(pdf_path: Path) -> str:
    doc = pymupdf.open(str(pdf_path))
    page = doc[0]
    mat = pymupdf.Matrix(2.0, 2.0)
    pix = page.get_pixmap(matrix=mat)
    img_bytes = pix.tobytes("png")
    img_b64 = base64.b64encode(img_bytes)
    doc.close()
    return img_b64.decode("utf-8")  

def extract_text_scan(pdf_path: Path) -> str:
    img_string = pdf_to_image(pdf_path)

    response = ollama.chat(
        model = OCR_MODEL,
        messages=[{
            "role": "user",
            "content": "Extrahiere mir den ganzen Text",
            "images": [img_string]
        }],
        think=False,
        options={
            "temperature": 0
        }
    )    
    return response.message.content

In [ ]:
# Define structured output and validation dict

from pydantic import BaseModel, Field, field_validator
from datetime import date, datetime

class Doctor(BaseModel):
    title:            str | None = Field(None, description="Academic or medical title, e.g. 'Dr.', 'Prim.', 'Univ.-Prof. Dr.'")
    first_name:       str | None = Field(None, description="First name of the doctor")
    last_name:        str | None = Field(None, description="Last name of the doctor")
    specialty:        str | None = Field(None, description="Medical specialty, e.g. 'Allgemeinmedizin', 'Innere Medizin', 'Dermatologie'")
    practice_name:    str | None = Field(None, description="Name of the medical practice if explicitly stated, not the doctor's name")
    practice_address: str | None = Field(None, description="Full address including street, number, postal code and city")
    phone:            str | None = Field(None, description="Phone number of the practice")
    email:            str | None = Field(None, description="Email address of the practice")
    uid:              str | None = Field(None, description="Austrian VAT number, format: ATU + 8 digits, e.g. 'ATU12345678'")

class Patient(BaseModel):
    first_name:              str  | None = Field(None, description="First name of the patient")
    last_name:               str  | None = Field(None, description="Last name of the patient")
    date_of_birth:           date | None = Field(None, description="Date of birth, format DD.MM.YYYY")
    social_security_number:  str  | None = Field(None, description="Austrian SVNR, format: '1234 150378'")

    @field_validator("date_of_birth", mode="before")
    @classmethod
    def parse_date_of_birth(cls, v):
        if v is None or isinstance(v, date):
            return v
        for fmt in ("%d.%m.%Y", "%Y-%m-%d", "%d/%m/%Y"):
            try:
                return datetime.strptime(v, fmt).date()
            except ValueError:
                continue
        return None
    
class Treatment(BaseModel):
    code:        str   | None = Field(None, description="Medical service code e.g. 'A00.0', 'L100'. Use null if not present.")
    description: str   | None = Field(None, description="Description of the medical service performed")
    amount:      float | None = Field(None, description="Cost of this treatment as decimal. Never include the total sum row.")

class Invoice(BaseModel):
    invoice_date:    date          | None = Field(None, description="Date the invoice was issued, format DD.MM.YYYY")
    invoice_number:  str           | None = Field(None, description="Invoice reference number, e.g. 'RE-1234-2025'")
    document_type:   str           | None = Field(None, description="Typically one of: 'Rechnung', 'Honorarnote', 'Arzthonorar', 'Privatrechnung'")
    doctor:          Doctor        | None = Field(None, description="Details of the issuing doctor")
    patient:         Patient       | None = Field(None, description="Details of the patient")
    diagnosis:       str           | None = Field(None, description="Medical diagnosis, include ICD-10 code if present, e.g. 'Gastritis (K29)'")
    treatments:      list[Treatment] | None = Field(None, description="List of treatments. Never include the total sum row as a treatment item.")
    total_amount:    float         | None = Field(None, description="Total invoice amount as decimal. ALWAYS take this value directly from the invoice. " \
                                                                    "It is always the last row of the treatment table. NEVER calculate it yourself.")
    payment_method:  str           | None = Field(None, description="ONLY 'cash' or 'bank transfer'. No other text, no IBAN, no account details.")
    iban:            str           | None = Field(None, description="IBAN if payment method is bank transfer")
    
    @field_validator("invoice_date", mode="before")
    @classmethod
    def parse_invoice_date(cls, v):
        if v is None or isinstance(v, date):
            return v
        for fmt in ("%d.%m.%Y", "%Y-%m-%d", "%d/%m/%Y"):
            try:
                return datetime.strptime(v, fmt).date()
            except ValueError:
                continue
        return None

def validate_invoice(invoice: Invoice) -> dict:
    flags = {}

    # Invoice
    flags["invoice_date_missing"] = invoice.invoice_date is None
    flags["invoice_number_missing"] = invoice.invoice_number is None
    flags["diagnosis_missing"] = invoice.diagnosis is None
    flags["treatments_missing"] = not invoice.treatments
    flags["total_amount_missing"]= invoice.total_amount is None

    # Doctor
    flags["doctor_first_name_missing"] = invoice.doctor is None or invoice.doctor.first_name is None
    flags["doctor_last_name_missing"] = invoice.doctor is None or invoice.doctor.last_name is None
    flags["doctor_specialty_missing"] = invoice.doctor is None or invoice.doctor.specialty is None
    flags["doctor_practice_address_missing"] = invoice.doctor is None or invoice.doctor.practice_address is None

    # Patient
    flags["patient_first_name_missing"] = invoice.patient is None or invoice.patient.first_name is None
    flags["patient_last_name_missing"] = invoice.patient is None or invoice.patient.last_name is None
    flags["patient_date_of_birth_missing"] = invoice.patient is None or invoice.patient.date_of_birth is None
    flags["patient_social_security_number_missing"] = invoice.patient is None or invoice.patient.social_security_number is None

    # Check if total amount of treatments is correct and if sum was included as treatment

    SUM_KEYWORDS = ["gesamt", "gesamtbetrag", "total", "honorar", "honorar gesamt", "summe", "sunne"]

    if invoice.treatments and invoice.total_amount:
        amounts = [treatment.amount for treatment in invoice.treatments if treatment.amount]
        calc = sum(amounts)
        last = amounts[-1]
        last_desc = invoice.treatments[-1].description.lower() if invoice.treatments[-1].description else ""      

        flags["total_mismatch"] = abs(calc - invoice.total_amount) > 0.05
        flags["last_treatment_equals_sum_of_others"] = calc - last == last
        flags["last_treatment_looks_like_sum"] = any(kw in last_desc for kw in SUM_KEYWORDS)

    else:
        flags["total_mismatch"] = False
        flags["last_treatment_equals_sum_of_others"] = False
        flags["last_treatment_looks_like_sum"] = False

    # Calculate score
    score = round(1 - sum(flags.values())/len(flags), 2)

    return {"score": score, "flags": flags}

In [ ]:
# Extract data from text
import json

SYSTEM_PROMPT = f"""Your are an expert in extracting information from Austrian private doctor's invoices (Wahlarztrechnungen)
Rules:
- If a field is not present in the invoice, use null
- NEVER calculate the total amount yourself. Always take it directly from the invoice
- NEVER include the total sum row as a treatment item
- If a treatment description looks like a misspelling of "Summe" or "Gesamt" (e.g. 'Sunne'), treat it as the total amount instead
- Date of birth format: DD.MM.YYYY as it appears on the invoice, e.g. '22.07.1965'
- Invoice date format: DD.MM.YYYY as it appears on the invoice
- payment_method: ONLY 'cash' or 'bank transfer', nothing else

Extract the following JSON strucutre:
{json.dumps(Invoice.model_json_schema(), indent=2)}
"""

def extract_structured_data(raw_text: str) -> tuple[Invoice | None, dict]:
    response = ollama.chat(
        model=LLM_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Here is the invoice text: {raw_text}"}
        ],
        think=False,
        format="json",
        #format=Invoice.model_json_schema(),
        options={"temperature": 0}        
    )
    #print("LLM Output: \n", response.message.content)

    try:
        
        invoice = Invoice.model_validate_json(response.message.content)
    except Exception as e:
        print(f"Validation error: {e}")
        return None, {}
    
    validation = validate_invoice(invoice)

    return invoice, validation

In [ ]:
# Pipeline

def process_pdf(pdf_path: Path)-> tuple[Invoice | None, dict]:
    pdf_type = detect_pdf_type(pdf_path)

    if pdf_type == "digital":
        raw_text = extract_text_digital(pdf_path)
    else:
        raw_text = extract_text_scan(pdf_path)
    
    invoice, validation = extract_structured_data(raw_text)

    return invoice, validation


# Test Pipeline
pdf_files = sorted(INVOICES_DIR.glob("*.pdf"))

pdf_digital = [path for path in pdf_files if "digital" in path.name][0]
pdf_scan = [path for path in pdf_files if "scan" in path.name][0]
pdf_hw = [path for path in pdf_files if "handwritten" in path.name][0]

invoice, validation = process_pdf(pdf_hw)

print("\nInvoice: \n", json.dumps(invoice.model_dump(mode="json"), indent=2, ensure_ascii=False))
print("\nValidation: \n", json.dumps(validation, indent=2))
